In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import math
from typing import Any, Dict, Optional, Tuple
from tqdm import tqdm
import ee
import pandas as pd
from google.colab import drive
drive.mount('/content/drive') #chemin
from google.colab import files


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=C9cQniWFPU3fMFktyJcECB6p7Qt7B3Ljri6m-w87WCU&tc=Qy5tIvHhCTNlsaHcQ-gnRzmAjEoPcfXYm8OwZs5hQDs&cc=xMXNsHLlsxerf24gLHBhtd9s1RlvNuTD6hH4PlxnWUY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6P6M6Dj-bmigu9LvuIGWl9DmG248BTR4o9-aLgj6CN6dZQhSMEq7g

Successfully saved authorization token.


In [ ]:
print(csv_madagascar['year'].unique())

[1997 2008 2011 2013 2016]


In [ ]:
# input data paths
CSV_PATH = '/content/drive/My Drive/madagascar.csv'
# band names
# # image export parameters
# PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
# SCALE = 30                # export resolution: 30m/px
# EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
# CHUNK_SIZE = 1      # set to a small number (<= 50) if Google Earth Engine reports memory errors; 
# !pwd

In [ ]:
BANDS      =  ["SR_B4","SR_B3","SR_B2","SR_B1","SR_B5","SR_B7"]
CRS        = 'EPSG:4326' # PLANE EPSG TO GET SQUARE IMAGES (THIS WILL REQUIRE SOME CONVERSION - SEE BELOW)
MARGIN     = 0.05

In [ ]:
csv_madagascar = pd.read_csv(CSV_PATH)

def create_bounding_box(lat, lon, margin=MARGIN):
    minx = lat-margin
    miny = lon-margin
    maxx = lat+margin
    maxy = lon+margin
    return [minx, miny, maxx, maxy]

csv_madagascar.drop(list(csv_madagascar.filter(regex = 'Unnamed')), axis = 1, inplace = True)
csv_madagascar['bounding_box'] = csv_madagascar.apply(lambda x: create_bounding_box(lat=x['lat'], lon=x['lon']), axis=1)
csv_madagascar.to_csv(CSV_PATH, sep=',')

In [ ]:
landsat_dates = [2001, 2015]

In [ ]:
country='madagascar'
collection = 'LANDSAT/LT05/C02/T1_L2'# Fall back to LANDSAT 5
for year in csv_madagascar['year'].unique():
    year = int(year)
    if year > landsat_dates[1]:
        collection ='LANDSAT/LC08/C02/T1_L2' # LANDSAT-8 available
    elif year > landsat_dates[0]:
        collection = 'LANDSAT/LE07/C02/T1_L2'# LANDSAT-7 available
    curr_year = csv_madagascar[ csv_madagascar['year']==year ]
    for index, row in tqdm(curr_year.iterrows()):
        tile_name = str(country)+'_'+str(year)+'_'+str(index)+'.tif'
        bb = row['bounding_box']
        xmin = float(bb[0])
        ymin = float(bb[1])
        xmax = float(bb[2])
        ymax = float(bb[3])
        region = ee.Geometry.BBox(ymin,xmin,ymax,xmax) # SWAP LAT/LON WITH EE 
        start_date = str(int(row['year'])-1)+'-01-01'
        end_date   = str(int(row['year'])+1)+'-12-31'
        data = ee.ImageCollection(collection)\
            .filterDate(start_date,end_date)\
            .filter(ee.Filter.lt('CLOUD_COVER',1))\
            .filterBounds(region)
        data = data.select(BANDS).mean()
        task = ee.batch.Export.image.toDrive(
            image=data,
            fileNamePrefix=str(index),
            folder='sentinel',
            region=region,
            scale=30,
            crs=CRS
        )
        task.start()

30it [00:12,  2.43it/s]
584it [05:04,  1.92it/s]
265it [01:17,  3.44it/s]
273it [01:50,  2.47it/s]
357it [02:09,  2.75it/s]
